In [2]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout, Reshape, Flatten
from keras.models import Model
from keras.layers import Input
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, GlobalAveragePooling2D

from keras.models import Sequential, ask_to_proceed_with_overwrite
from keras.layers import Dense
from keras.layers.core import Activation, Dense, Lambda
from keras.layers import SimpleRNN, LSTM,  GRU,SeparableConvolution2D
from keras.callbacks import ModelCheckpoint,EarlyStopping



import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers.core import Activation, Dense
import tensorflow as tf
import glob

import scipy
import numpy as np
from PIL import Image
import random
import pandas as pd
import random

In [21]:
number_of_brands=7
inputsize=224
max_frames=15

In [6]:
# def getModel( output_dim ):
#     ''' 
#         * output_dim: the number of classes (int)
        
#         * return: compiled model (keras.engine.training.Model)
#     '''
#     vgg_model = VGG16( weights='imagenet', include_top=True )
#     vgg_out = vgg_model.layers[-2].output #Last FC layer's output  
    
#     softmax_layer = Dense(output_dim, activation='softmax')(vgg_out)#None #Create softmax layer taking input as vgg_out
#     #Create new transfer learning model
#     tl_model = Model( input=vgg_model.input, output=softmax_layer )

#     #Freeze all layers of VGG16 and Compile the model
#     #Confirm the model is appropriate

#     return tl_model

def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    #vgg_model = VGG16( weights='imagenet', include_top=False )
    vgg_model = VGG16(weights='imagenet', include_top=False)

    for layer in vgg_model.layers[:-3]:
        layer.trainable = False

    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(output_dim ,activation='softmax')(x)

# this is the model we will train
    model = Model(input=vgg_model.input, output=predictions)

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate

    return model

   

def getModel2( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    #vgg_model = VGG16( weights='imagenet', include_top=False )
    vgg_model = VGG16(weights='imagenet', include_top=False)

#     for layer in vgg_model.layers:
#         layer.trainable = False

    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    
    predictions = Dense(output_dim ,activation='softmax')(x)

# this is the model we will train
    model = Model(input=vgg_model.input, output=predictions)

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate

    return model    

def load_model(weights_path, output_dim):

    model = getModel2( output_dim ) 
    for k,layer in list(model.layers_by_depth.items())[:]:
        layer[0].trainable = False
#     for layer in model.layers:
#         layer.trainable = False
        
    model.load_weights(weights_path)
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy', 'categorical_crossentropy'])   
    return model

In [21]:
m =  getModel( 5 ) 

In [19]:
m.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, None, None, 64 1792        input_6[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, None, None, 64 36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, None, None, 64 0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [7]:
trained_model = load_model("weights-improvement-CPU-VGG16-LOGO7-02-0.8741.hdf5", 7)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:67: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [8]:
trained_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [5]:
trained_model.layers.pop()

In [9]:
model_rnn3=Sequential()

model_rnn3.add(TimeDistributed(trained_model, input_shape=(max_frames,224,224,3)))
model_rnn3.add(GRU(output_dim=100,return_sequences=True))
model_rnn3.add(Dropout(.2))
model_rnn3.add(GRU(output_dim=50,return_sequences=False))
model_rnn3.add(Dropout(.2))
model_rnn3.add(Dense(number_of_brands,activation='softmax'))
model_rnn3.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy', 'categorical_crossentropy'])



def load_rnn_model(weights):
    model_rnn3=Sequential()

    model_rnn3.add(TimeDistributed(trained_model, input_shape=(max_frames,224,224,3)))
    model_rnn3.add(GRU(output_dim=100,return_sequences=True))
    model_rnn3.add(GRU(output_dim=50,return_sequences=False))
    model_rnn3.add(Dropout(.2))
    model_rnn3.add(Dense(number_of_brands,activation='softmax'))
    model_rnn3.load_weights(weights)

    model_rnn3.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy', 'categorical_crossentropy'])

    return model_rnn3

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(units=100, return_sequences=True)`
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(units=50, return_sequences=False)`


In [9]:
model_rnn3.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
timedistributed_1 (TimeDistribute(None, 20, 4)         0           timedistributed_input_1[0][0]    
____________________________________________________________________________________________________
gru_1 (GRU)                      (None, 20, 100)       31500       timedistributed_1[0][0]          
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 20, 100)       0           gru_1[0][0]                      
____________________________________________________________________________________________________
gru_2 (GRU)                      (None, 50)            22650       dropout_1[0][0]                  
___________________________________________________________________________________________

In [7]:
## Load data

In [17]:
labels_dict = { 'cocacola': 0, 'nike': 3, 'pepsi': 1, 'noise':4}
labels_dict = {"cocacola":0, "loreal":1,"louisvuitton":2,"nike":3,"pepsi":4,"samsung":5, "noise":6}

In [11]:
from idxs import *

ImportError: No module named 'idxs'

In [20]:
number_of_brands

4

In [18]:
import re
import math

max_frames=10

def closest_number_idx(l, num):
    return l.index(min(l, key=lambda x:abs(x-num)))

def get_closest_frames(files, initial_frame):
    frames_num = [int(re.search("\d\d\d\d", x).group()) for x in files]
    closest_idx = closest_number_idx(frames_num, initial_frame)
    #new_frames = files[closest_idx-2: closest_idx+3]
    new_frames = files[closest_idx-3: closest_idx+7]
    #new_frames = files[closest_idx: closest_idx+max_frames]


    #new_frames = files[closest_idx-5: closest_idx+10]


    return new_frames

def load_frames(label, idxs, max_frames):
    tmp_X = []
    tmp_y = []
    for path, initial_frame in idxs:
        path = "../code" + path[1:] + "/*"
        files = sorted(glob.glob(path))
        #print len(files)
        row_X = []
        closest_frames = get_closest_frames(files, initial_frame)
        #print closest_frames
        y_row = np.zeros(number_of_brands)
        y_row[label] = 1
        for f in closest_frames:
            img = np.asarray(Image.open(f))/255.
            row_X.append(img)
        if len(row_X) != max_frames:
            continue
        row_X = np.array(row_X)
        #print row_X.shape
        tmp_X.append(row_X)
        tmp_y.append(y_row)
    return (tmp_y, tmp_X)

def load_new_frames(path):
    t = pd.read_csv("top_frames_LOGO7_frames10.csv", header=None).as_matrix()
    tmp_X = []
    tmp_y = []
    for row in t:
        label = labels_dict[row[0].split("/")[-2].lower()]
        all_frames = sorted(glob.glob(row[0] + "/*.jpg"))
        idx_frames = all_frames.index(row[1])
        selected_frames = all_frames[idx_frames:idx_frames+max_frames]
        row_X = []
        y_row = np.zeros(number_of_brands)
        y_row[label] = 1
        for f in selected_frames:
            img = np.asarray(Image.open(f))/255.
            row_X.append(img)
        row_X = np.array(row_X)
        #print row_X.shape
        tmp_X.append(row_X)
        tmp_y.append(y_row)
    return (tmp_y, tmp_X)

In [19]:
def load_frames_test_frames(path,i):
    frames = sorted(glob.glob(path+"*.jpg"))[10:10+max_frames]
    print(frames)
    row_X = []
    for f in frames:
        img = np.asarray(Image.open(f))/255.
        row_X.append(img)
    return np.array(row_X)

In [13]:
cocacola_idxs

[('./cocacola_processed/cocacola/10', 901),
 ('./cocacola_processed/cocacola/55', 201),
 ('./cocacola_processed/cocacola/147', 147),
 ('./cocacola_processed/cocacola/21', 703),
 ('./cocacola_processed/cocacola/87', 743),
 ('./cocacola_processed/cocacola/4', 509),
 ('./cocacola_processed/cocacola/126', 14),
 ('./cocacola_processed/cocacola/136', 320),
 ('./cocacola_processed/cocacola/25', 133),
 ('./cocacola_processed/cocacola/33', 528),
 ('./cocacola_processed/cocacola/100', 121),
 ('./cocacola_processed/cocacola/115', 493),
 ('./cocacola_processed/cocacola/15', 22),
 ('./cocacola_processed/cocacola/36', 95),
 ('./cocacola_processed/cocacola/65', 184),
 ('./cocacola_processed/cocacola/66', 657),
 ('./cocacola_processed/cocacola/121', 674),
 ('./cocacola_processed/cocacola/75', 0),
 ('./cocacola_processed/cocacola/84', 56),
 ('./cocacola_processed/cocacola/150', 320),
 ('./cocacola_processed/cocacola/60', 106),
 ('./cocacola_processed/cocacola/117', 489),
 ('./cocacola_processed/cocacol

In [22]:
y1,X1 = load_new_frames("top_frames_LOGO7_frames10.csv")

In [30]:
y6_X6 = np.array([load_frames_test_frames("/Users/francescoferrari/Downloads/junk_processed/"+str(x)+"/",1) for x in range(18) if len(load_frames_test_frames("/Users/francescoferrari/Downloads/junk_processed/"+str(x)+"/",1)) == max_frames]
                 +[load_frames_test_frames("/Users/francescoferrari/Downloads/junk_processed/"+str(x)+"/",6) for x in range(18) if len(load_frames_test_frames("/Users/francescoferrari/Downloads/junk_processed/"+str(x)+"/",6)) == max_frames]
                +[load_frames_test_frames("/Users/francescoferrari/Downloads/junk_processed/"+str(x)+"/",11) for x in range(18) if len(load_frames_test_frames("/Users/francescoferrari/Downloads/junk_processed/"+str(x)+"/",11)) > 0]
                )

['/Users/francescoferrari/Downloads/junk_processed/0/f_0011.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0012.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0013.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0014.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0015.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0016.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0017.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0018.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0019.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0020.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0021.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0022.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0023.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0024.jpg', '/Users/francescoferrari/Downloads/junk_processed/0/f_0025.jpg']
['/Users/francescoferrar

In [10]:
y1,X1 = load_frames(0, cocacola_idxs, max_frames)
y2,X2 = load_frames(1, pepsi_idxs, max_frames)
y3,X3 = load_frames(2, apple_idxs, max_frames)
#y4,X4 = load_frames(3, starbucks_idxs, max_frames)
y5,X5 = load_frames(3, nike_idxs, max_frames)
y6_X6 = np.array([load_frames_test_frames("junk_processed/"+str(x)+"/",1) for x in range(18) if len(load_frames_test_frames("junk_processed/"+str(x)+"/",1)) == max_frames]
                 #[load_frames_test_frames("junk_processed/"+str(x)+"/",6) for x in range(18) if len(load_frames_test_frames("junk_processed/"+str(x)+"/",6)) == max_frames]
                #[load_frames_test_frames("junk_processed/"+str(x)+"/",11) for x in range(18) if len(load_frames_test_frames("junk_processed/"+str(x)+"/",11)) > 0]
                )

['junk_processed/0/f_0011.jpg', 'junk_processed/0/f_0012.jpg', 'junk_processed/0/f_0013.jpg', 'junk_processed/0/f_0014.jpg', 'junk_processed/0/f_0015.jpg', 'junk_processed/0/f_0016.jpg', 'junk_processed/0/f_0017.jpg', 'junk_processed/0/f_0018.jpg', 'junk_processed/0/f_0019.jpg', 'junk_processed/0/f_0020.jpg']
['junk_processed/0/f_0011.jpg', 'junk_processed/0/f_0012.jpg', 'junk_processed/0/f_0013.jpg', 'junk_processed/0/f_0014.jpg', 'junk_processed/0/f_0015.jpg', 'junk_processed/0/f_0016.jpg', 'junk_processed/0/f_0017.jpg', 'junk_processed/0/f_0018.jpg', 'junk_processed/0/f_0019.jpg', 'junk_processed/0/f_0020.jpg']
['junk_processed/1/f_0011.jpg', 'junk_processed/1/f_0012.jpg', 'junk_processed/1/f_0013.jpg', 'junk_processed/1/f_0014.jpg', 'junk_processed/1/f_0015.jpg', 'junk_processed/1/f_0016.jpg', 'junk_processed/1/f_0017.jpg', 'junk_processed/1/f_0018.jpg', 'junk_processed/1/f_0019.jpg', 'junk_processed/1/f_0020.jpg']
['junk_processed/1/f_0011.jpg', 'junk_processed/1/f_0012.jpg', 'jun

In [ ]:
[load_frames_test_frames("junk_processed/"+str(x)+"/") for x in range(18) if len(load_frames_test_frames("junk_processed/"+str(x)+"/")) > 0]

In [36]:
X5.shape

AttributeError: 'list' object has no attribute 'shape'

In [31]:
y6 = (np.zeros((len(y6_X6), number_of_brands)))
y6[:,number_of_brands-1] = 1
y6 = list(y6)
X6 = list(y6_X6)

In [32]:
y6

[array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0

In [67]:
X = X1+X2+X3+X5+X6
y = y1+y2+y3+y5+y6

NameError: name 'X2' is not defined

In [33]:
X = X1+X6
y = y1+y6

In [34]:
len(X)

729

In [37]:
data = list(zip(y, X))

In [38]:
random.shuffle(data)

In [39]:
X = [x[1] for x in data]
y = [x[0] for x in data]

In [40]:
X = np.array(X)
y = np.array(y)

In [41]:
X.shape

(729, 15, 224, 224, 3)

In [42]:
x_mean = X.mean()

In [43]:
x_mean

0.44021639613396263

In [44]:
for i in range(len(X)):
    X[i] -= x_mean

In [46]:
X[3]

array([[[[-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483],
         ..., 
         [-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483]],

        [[-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483],
         ..., 
         [-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483]],

        [[-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483],
         ..., 
         [-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483],
         [-0.43629483, -0.43629483, -0.43629483]],

        ..., 
        [[-0.43629483, -0.43629483,

In [49]:
limit = int(len(X)/10*8)

In [50]:
X_train = X[:limit]
X_val = X[limit:]
y_train = y[:limit]
y_val = y[limit:]

In [51]:
y_train.shape

(583, 7)

In [31]:
labels_dict = {'apple': 2, 'cocacola': 0, 'nike': 4, 'pepsi': 1, 'starbucks': 3, 'noise':5}

In [52]:
model_rnn3=Sequential()

model_rnn3.add(TimeDistributed(trained_model, input_shape=(max_frames,224,224,3)))
model_rnn3.add(GRU(output_dim=200,return_sequences=True))
model_rnn3.add(GRU(output_dim=100,return_sequences=True))
model_rnn3.add(GRU(output_dim=50,return_sequences=False))
model_rnn3.add(Dropout(.2))
model_rnn3.add(Dense(number_of_brands,activation='softmax'))
#model_rnn3.add(Lambda(lambda x: x / 2.0))
model_rnn3.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy', 'categorical_crossentropy'])

class_weight = {0 : 100.,
    1: 100.,
    2: 100.,
    3:100.,
    4:100.;
    5:100.,
    6:1.}

SyntaxError: invalid syntax (<ipython-input-52-74b84e91fdcf>, line 16)

In [82]:
filepath="RNN-LOGO4-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    
earlyStopping=EarlyStopping(monitor='val_acc', patience=4, verbose=0, mode='auto')
callbacks_list = [earlyStopping, checkpoint]

In [ ]:
m = model_rnn3.fit(X_train, y_train, validation_data=(X_val, y_val),
              batch_size=16, nb_epoch=10, verbose=1, callbacks=callbacks_list, class_weight=class_weight)

Train on 96 samples, validate on 32 samples
Epoch 1/10


In [36]:
max_frames

10

In [49]:
from collections import *
Counter([np.argmax(x) for x in y_train])

Counter({0: 17, 1: 33, 2: 43, 3: 3, 4: 64})

In [61]:
import plotly.plotly as py
import plotly.graph_objs as go

# Add data
month = range(200)
n_100_train = h["acc"]
n_100_val = h["val_acc"]

# Create and sth150.history["loss"]yle traces
trace0 = go.Scatter(
    x = month,
    y = n_100_train,
    name = 'train',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)
trace1 = go.Scatter(
    x = month,
    y = n_100_val,
    name = 'val',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 4,)
)

data = [trace0, trace1]

# Edit the layout
layout = dict(title = 'Accuracy vs Epochs - RNN Accuracy over 5 frames (dense layer)',
              xaxis = dict(title = 'Epochs'),
              yaxis = dict(title = 'Accuracy'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='cse253/final_project/fig2_rnn_denselayer')

In [13]:
## Testing

In [8]:
rnn_model = load_rnn_model('RNN-05-0.8704.hdf5')

In [56]:
test_frames = np.array([load_frames_test_frames("junk_processed/"+str(x)+"/") for x in range(18) if len(load_frames_test_frames("junk_processed/"+str(x)+"/")) > 0])
ids = np.array([x for x in range(18) if len(load_frames_test_frames("junk_processed/"+str(x)+"/")) > 0])
test_frames = test_frames - x_mean

['junk_processed/0/f_0011.jpg', 'junk_processed/0/f_0012.jpg', 'junk_processed/0/f_0013.jpg', 'junk_processed/0/f_0014.jpg', 'junk_processed/0/f_0015.jpg']
['junk_processed/0/f_0011.jpg', 'junk_processed/0/f_0012.jpg', 'junk_processed/0/f_0013.jpg', 'junk_processed/0/f_0014.jpg', 'junk_processed/0/f_0015.jpg']
['junk_processed/1/f_0011.jpg', 'junk_processed/1/f_0012.jpg', 'junk_processed/1/f_0013.jpg', 'junk_processed/1/f_0014.jpg', 'junk_processed/1/f_0015.jpg']
['junk_processed/1/f_0011.jpg', 'junk_processed/1/f_0012.jpg', 'junk_processed/1/f_0013.jpg', 'junk_processed/1/f_0014.jpg', 'junk_processed/1/f_0015.jpg']
['junk_processed/2/f_0011.jpg', 'junk_processed/2/f_0012.jpg', 'junk_processed/2/f_0013.jpg', 'junk_processed/2/f_0014.jpg', 'junk_processed/2/f_0015.jpg']
['junk_processed/2/f_0011.jpg', 'junk_processed/2/f_0012.jpg', 'junk_processed/2/f_0013.jpg', 'junk_processed/2/f_0014.jpg', 'junk_processed/2/f_0015.jpg']
['junk_processed/3/f_0011.jpg', 'junk_processed/3/f_0012.jpg', '

In [39]:
test_frames[0]

array([ array([[[[ 0.        ,  0.00392157,  0.02745098],
         [ 0.        ,  0.00392157,  0.03921569],
         [ 0.        ,  0.00392157,  0.04705882],
         ..., 
         [ 0.02745098,  0.02352941,  0.01568627],
         [ 0.02352941,  0.01960784,  0.01176471],
         [ 0.02745098,  0.02352941,  0.01568627]],

        [[ 0.        ,  0.01568627,  0.03921569],
         [ 0.        ,  0.01176471,  0.04705882],
         [ 0.        ,  0.01568627,  0.05882353],
         ..., 
         [ 0.02745098,  0.02352941,  0.01568627],
         [ 0.02745098,  0.02352941,  0.01568627],
         [ 0.02745098,  0.02352941,  0.01568627]],

        [[ 0.00392157,  0.02352941,  0.04705882],
         [ 0.00784314,  0.02352941,  0.05882353],
         [ 0.00784314,  0.02352941,  0.06666667],
         ..., 
         [ 0.03137255,  0.02745098,  0.01960784],
         [ 0.02745098,  0.02352941,  0.01568627],
         [ 0.03137255,  0.02745098,  0.01960784]],

        ..., 
        [[ 0.01568627,  0.0

In [57]:
zip(ids,rnn_model.predict(test_frames))

[(0,
  array([ 0.02990815,  0.87141144,  0.03884432,  0.01783795,  0.04199824], dtype=float32)),
 (1,
  array([ 0.05284804,  0.85617864,  0.02813121,  0.01266972,  0.05017231], dtype=float32)),
 (2,
  array([ 0.02113564,  0.79115915,  0.13195661,  0.01407295,  0.04167565], dtype=float32)),
 (3,
  array([ 0.04125538,  0.66306341,  0.25089926,  0.01138087,  0.03340105], dtype=float32)),
 (4,
  array([ 0.05142019,  0.26781598,  0.6499601 ,  0.00649989,  0.0243038 ], dtype=float32)),
 (5,
  array([ 0.00897984,  0.04401732,  0.93387884,  0.00136466,  0.01175937], dtype=float32)),
 (7,
  array([ 0.00194785,  0.00265935,  0.03473446,  0.00238633,  0.95827204], dtype=float32)),
 (9,
  array([ 0.00781901,  0.04496627,  0.93763733,  0.00115217,  0.00842521], dtype=float32)),
 (10,
  array([ 0.04500297,  0.73338723,  0.08002244,  0.02114916,  0.12043814], dtype=float32)),
 (11,
  array([ 0.01730857,  0.0263928 ,  0.63424027,  0.00629761,  0.3157607 ], dtype=float32)),
 (12,
  array([ 0.01106732, 

In [29]:
labels_dict

{'apple': 2, 'cocacola': 0, 'nike': 4, 'pepsi': 1, 'starbucks': 3}

In [16]:
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot, plot

SVG(model_to_dot(rnn_model).create(prog='dot', format='svg'))#plot_model(rnn_model, to_file='model.png')

In [9]:
from keras import utils

In [19]:
plot(rnn_model, to_file='rnn.png')
plot(trained_model, to_file='cnn.png')



In [12]:
dir(utils.layer_utils)

['Activation',
 'ActivityRegularization',
 'AtrousConv1D',
 'AtrousConv2D',
 'AtrousConvolution1D',
 'AtrousConvolution2D',
 'AveragePooling1D',
 'AveragePooling2D',
 'AveragePooling3D',
 'BatchNormalization',
 'Bidirectional',
 'Conv1D',
 'Conv2D',
 'Conv3D',
 'ConvLSTM2D',
 'ConvRecurrent2D',
 'Convolution1D',
 'Convolution2D',
 'Convolution3D',
 'Cropping1D',
 'Cropping2D',
 'Cropping3D',
 'Deconv2D',
 'Deconvolution2D',
 'Dense',
 'Dropout',
 'ELU',
 'Embedding',
 'Flatten',
 'GRU',
 'GaussianDropout',
 'GaussianNoise',
 'GlobalAveragePooling1D',
 'GlobalAveragePooling2D',
 'GlobalAveragePooling3D',
 'GlobalMaxPooling1D',
 'GlobalMaxPooling2D',
 'GlobalMaxPooling3D',
 'Highway',
 'Input',
 'InputLayer',
 'InputSpec',
 'K',
 'LSTM',
 'Lambda',
 'Layer',
 'LeakyReLU',
 'LocallyConnected1D',
 'LocallyConnected2D',
 'Masking',
 'MaxPooling1D',
 'MaxPooling2D',
 'MaxPooling3D',
 'MaxoutDense',
 'Merge',
 'Model',
 'PReLU',
 'ParametricSoftplus',
 'Permute',
 'Recurrent',
 'RepeatVector'